# SCRIPT DE GENERACIÓN Y PARTICIÓN DEL DATASET

Este script organiza las imágenes aumentadas en una estructura de directorios estándar para entrenamiento de Deep Learning. Divide los datos aleatoriamente en tres subconjuntos:
- Train (Entrenamiento): Para ajustar los pesos del modelo.
- Val (Validación): Para ajustar hiperparámetros y monitorear el overfitting.
- Test (Prueba): Para la evaluación final con datos inéditos.


In [ ]:
import os
import shutil
import random

# === CONFIGURACIÓN DE RUTAS Y PARÁMETROS ===
# Directorios de origen (donde guardaste las imágenes aumentadas)
IMAGES_DIR = 'Aum_images'
MASKS_DIR = 'Aum_masks'

# Directorio base de salida (se creará si no existe)
OUTPUT_BASE = 'dataset'

# Proporciones de división del dataset (Debe sumar 1.0)
# 70% Entrenamiento, 25% Validación, 5% Prueba
SPLIT_RATIOS = {'train': 0.7, 'val': 0.25, 'test': 0.05}

def split_dataset():
    """
    Ejecuta la partición aleatoria y copia los archivos a las carpetas de destino.
    """
    
    # 1. Obtener y barajar la lista de archivos
    # -------------------------------------------------------------------------
    # Filtramos solo archivos de imagen para evitar archivos ocultos o de sistema
    image_files = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
    
    # Establecemos una semilla (seed) para que la mezcla sea reproducible.
    # Si ejecutas este script dos veces, la división será idéntica.
    random.seed(42) 
    random.shuffle(image_files)

    # 2. Calcular los índices de corte
    # -------------------------------------------------------------------------
    total = len(image_files)
    num_train = int(total * SPLIT_RATIOS['train'])
    num_val = int(total * SPLIT_RATIOS['val'])
    # El resto se asigna a test para asegurar que no se pierda ningún archivo por redondeo
    num_test = total - num_train - num_val

    print(f"Total imágenes encontradas: {total}")
    print(f"Distribución -> Train: {num_train}, Val: {num_val}, Test: {num_test}")

    # Diccionario con las listas de archivos para cada set
    splits = {
        'train': image_files[:num_train],
        'val': image_files[num_train:num_train + num_val],
        'test': image_files[num_train + num_val:]
    }

    # 3. Crear estructura de carpetas y copiar archivos
    # -------------------------------------------------------------------------
    for split_name, files in splits.items():
        print(f"\nProcesando conjunto: {split_name.upper()}...")
        
        # Crear directorios: dataset/train/images, dataset/train/masks, etc.
        # makedirs con exist_ok=True evita errores si la carpeta ya existe
        os.makedirs(os.path.join(OUTPUT_BASE, split_name, 'images'), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_BASE, split_name, 'masks'), exist_ok=True)

        for fname in files:
            # --- Procesamiento de la IMAGEN ---
            src_img = os.path.join(IMAGES_DIR, fname)
            dst_img = os.path.join(OUTPUT_BASE, split_name, 'images', fname)
            
            # Copiar imagen al destino
            shutil.copyfile(src_img, dst_img)

            # --- Procesamiento de la MÁSCARA ---
            # Asumimos que la máscara tiene el mismo nombre base que la imagen
            # pero con extensión .png (formato sin pérdida para etiquetas)
            base_name = os.path.splitext(fname)[0]
            
            # NOTA: Si tus máscaras tienen otro sufijo (ej: _mask.png), ajusta esta línea
            mask_name = base_name + '.png' 

            src_mask = os.path.join(MASKS_DIR, mask_name)
            dst_mask = os.path.join(OUTPUT_BASE, split_name, 'masks', mask_name)

            # Verificación de seguridad: ¿Existe la máscara correspondiente?
            if not os.path.exists(src_mask):
                print(f" [ADVERTENCIA] Máscara no encontrada para {fname}. Se omite este par.")
                # Si falta la máscara, podríamos borrar la imagen copiada para no ensuciar el dataset
                # os.remove(dst_img) 
                continue

            # Copiar máscara al destino
            shutil.copyfile(src_mask, dst_mask)

    print(f"\nProceso finalizado. Dataset generado en la carpeta '{OUTPUT_BASE}'.")

# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    # Verificar que los directorios de entrada existan antes de empezar
    if os.path.exists(IMAGES_DIR) and os.path.exists(MASKS_DIR):
        split_dataset()
    else:
        print("Error: No se encuentran los directorios de entrada (Aum_images / Aum_masks).")
        print("Ejecuta primero el script de Data Augmentation.")